In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Audio Processing Libraries
import librosa
from scipy import signal
#For Playing Audio
import IPython.display as ipd
#Array Processing
import numpy as np
#Data Visualization
import matplotlib.pyplot as plt

In [3]:
import os
import shutil
import random

In [4]:
!git clone https://github.com/ultralytics/yolov5.git 
%cd yolov5/

Cloning into 'yolov5'...
remote: Enumerating objects: 14156, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 14156 (delta 56), reused 74 (delta 43), pack-reused 14060
Receiving objects: 100% (14156/14156), 13.60 MiB | 27.20 MiB/s, done.
Resolving deltas: 100% (9712/9712), done.
/content/yolov5


In [5]:
import tensorflow

In [6]:
import keras

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path='/content/drive/MyDrive/Audio_classification/emergency.wav'
emergency,sample_rate    = librosa.load(path, sr = 16000)
path='/content/drive/MyDrive/Audio_classification/non emergency new.wav'
non_emergency,sample_rate= librosa.load(path, sr =16000)

In [8]:
duration1 = librosa.get_duration(emergency,sr=16000)
duration2 = librosa.get_duration(non_emergency,sr=16000)
print("Duration of an emergency and Non Emergency (in min):",duration1/60,duration2/60)

Duration of an emergency and Non Emergency (in min): 22.920290625 8.245784375000001


In [9]:
def prepare_data(samples, num_of_samples=32000, num_of_common=16000):
    data=[]
    for offset in range(0,len(samples),num_of_common):
        start = offset
        end = offset + num_of_samples
        chunk = samples[start:end]
        print(end)
        if(len(chunk)==32000):
            data.append(chunk)
        else:
            print(len(chunk))
    return data
        

In [10]:
emergency = prepare_data(emergency)
non_emergency = prepare_data(non_emergency)


32000
48000
64000
80000
96000
112000
128000
144000
160000
176000
192000
208000
224000
240000
256000
272000
288000
304000
320000
336000
352000
368000
384000
400000
416000
432000
448000
464000
480000
496000
512000
528000
544000
560000
576000
592000
608000
624000
640000
656000
672000
688000
704000
720000
736000
752000
768000
784000
800000
816000
832000
848000
864000
880000
896000
912000
928000
944000
960000
976000
992000
1008000
1024000
1040000
1056000
1072000
1088000
1104000
1120000
1136000
1152000
1168000
1184000
1200000
1216000
1232000
1248000
1264000
1280000
1296000
1312000
1328000
1344000
1360000
1376000
1392000
1408000
1424000
1440000
1456000
1472000
1488000
1504000
1520000
1536000
1552000
1568000
1584000
1600000
1616000
1632000
1648000
1664000
1680000
1696000
1712000
1728000
1744000
1760000
1776000
1792000
1808000
1824000
1840000
1856000
1872000
1888000
1904000
1920000
1936000
1952000
1968000
1984000
2000000
2016000
2032000
2048000
2064000
2080000
2096000
2112000
2128000
2144000
21

In [11]:
print(emergency) 

[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0.        , 0.        , 0.        , ..., 0.05222268, 0.05082379,
       0.03745775], dtype=float32), array([0.        , 0.        , 0.        , ..., 0.0553952 , 0.05667262,
       0.04818304], dtype=float32), array([ 0.02421285,  0.01220857,  0.00034687, ..., -0.04591297,
       -0.00844002,  0.01474224], dtype=float32), array([ 0.03142681,  0.02424584,  0.01440367, ...,  0.06073397,
        0.03552166, -0.00758342], dtype=float32), array([0.01480003, 0.01822437, 0.02368303, ..., 0.05732322, 0.03041729,
       0.01397711], dtype=float32), array([-0.04675494, -0.05688571, -0.03370411, ...,  0.10176856,
        0.03685023, -0.04052664], dtype=float32), array([ 0.01706938,  0.01183748, -0.00305859, ..., -0.22154033,
    

In [12]:
print("No. of Chunks of Emergency and Non Emergency:",len(emergency),len(non_emergency))

No. of Chunks of Emergency and Non Emergency: 1374 493


In [13]:
audio = np.concatenate([emergency,non_emergency])
labels1 = np.zeros(len(emergency))
labels2 = np.ones(len(non_emergency))
labels = np.concatenate([labels1,labels2])
print(audio.shape)
print(labels.shape)

(1867, 32000)
(1867,)


In [14]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(audio),np.array(labels),
                                             stratify=labels,test_size = 0.1,
                                                   random_state=777,shuffle=True)

In [15]:
x_tr_features  = x_tr.reshape(len(x_tr),-1,1)
x_val_features = x_val.reshape(len(x_val),-1,1)
print("Reshaped Array Size",x_tr_features.shape)

Reshaped Array Size (1680, 32000, 1)


In [16]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras import backend as K
def cnn(x_tr):
    K.clear_session()
    inputs = Input(shape=(x_tr.shape[1],x_tr.shape[2]))
    #First Conv1D layer
    conv = Conv1D(8, 13, padding='same', activation='relu')(inputs)
    conv = Dropout(0.3)(conv)
    conv = MaxPooling1D(2)(conv)
    #Second Conv1D layer
    conv = Conv1D(16, 11, padding='same', activation='relu')(conv)
    conv = Dropout(0.3)(conv)
    conv = MaxPooling1D(2)(conv)
    #MaxPooling 1D
    conv = GlobalMaxPool1D()(conv)
    #Dense Layer 
    conv = Dense(16, activation='relu')(conv)
    outputs = Dense(1,activation='sigmoid')(conv)
    model = Model(inputs, outputs)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
    model_checkpoint = ModelCheckpoint('best_model.hdf5', monitor='val_acc',
                                 verbose=1, save_best_only=True, mode='max')
    return model, model_checkpoint
model, model_checkpoint = cnn(x_tr_features)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32000, 1)]        0         
                                                                 
 conv1d (Conv1D)             (None, 32000, 8)          112       
                                                                 
 dropout (Dropout)           (None, 32000, 8)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16000, 8)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 16000, 16)         1424      
                                                                 
 dropout_1 (Dropout)         (None, 16000, 16)         0         
                                                             

In [45]:
history=model.fit(x_tr_features, y_tr ,epochs=10,       callbacks=[model_checkpoint], 
batch_size=32, validation_data=(x_val_features,y_val)) 

Epoch 1/10
 1/53 [..............................] - ETA: 2:08 - loss: 0.5966 - acc: 0.7500

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files
files.download('/content/best_model.hdf5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
model.load_weights('/content/drive/MyDrive/best_model.hdf5')

In [19]:
_, acc = model.evaluate(x_val_features,y_val)
print("Validation Accuracy:",acc)

6/6 [==============================] - 2s 187ms/step - loss: 0.3449 - acc: 0.8449
Validation Accuracy: 0.8449198007583618


In [20]:
ind=35
test_audio = x_val[ind]
ipd.Audio(test_audio,rate=16000)
feature = x_val_features[ind]
prob = model.predict(feature.reshape(1,-1,1))
if (prob[0][0] < 0.5 ):
  pred='emergency'
else:
  pred='non emergency'
print("Prediction:",pred)

1/1 [==============================] - 0s 131ms/step
Prediction: emergency


In [21]:
path='/content/drive/MyDrive/Audio_classification/reduced.wav'
emergency_test,sample_rate    = librosa.load(path, sr = 16000)

In [28]:
path='/content/drive/MyDrive/Audio_classification/non emergency new.wav'
emergency_test,sample_rate    = librosa.load(path, sr = 16000)

In [29]:
def prepare_data_for_test(samples, num_of_samples=32000, num_of_common=16000):
    data_1=[]
    for offset in range(0,len(samples),num_of_common):
        start_1 = offset
        end_1 = offset + num_of_samples
        chunk_1 = samples[start_1:end_1]
        print(end_1)
        if(len(chunk_1)==32000):
            data_1.append(chunk_1)
        else:
            print(len(chunk_1))
    return data_1
        

In [30]:
trial_list = prepare_data_for_test(emergency_test)

32000
48000
64000
80000
96000
112000
128000
144000
160000
176000
192000
208000
224000
240000
256000
272000
288000
304000
320000
336000
352000
368000
384000
400000
416000
432000
448000
464000
480000
496000
512000
528000
544000
560000
576000
592000
608000
624000
640000
656000
672000
688000
704000
720000
736000
752000
768000
784000
800000
816000
832000
848000
864000
880000
896000
912000
928000
944000
960000
976000
992000
1008000
1024000
1040000
1056000
1072000
1088000
1104000
1120000
1136000
1152000
1168000
1184000
1200000
1216000
1232000
1248000
1264000
1280000
1296000
1312000
1328000
1344000
1360000
1376000
1392000
1408000
1424000
1440000
1456000
1472000
1488000
1504000
1520000
1536000
1552000
1568000
1584000
1600000
1616000
1632000
1648000
1664000
1680000
1696000
1712000
1728000
1744000
1760000
1776000
1792000
1808000
1824000
1840000
1856000
1872000
1888000
1904000
1920000
1936000
1952000
1968000
1984000
2000000
2016000
2032000
2048000
2064000
2080000
2096000
2112000
2128000
2144000
21

In [31]:
i = np.array([trial_list])

In [32]:
j = i.reshape(i.shape[1],i.shape[-1],1)

In [33]:
ind_1=0
# test_audio = x_val[ind]
# ipd.Audio(test_audio,rate=16000)
feature = j[ind_1]
prob = model.predict(feature.reshape(1,-1,1))
if (prob[0][0] < 0.5 ):
  pred='emergency'
else:
  pred='non emergency'
print("Prediction:",pred)

1/1 [==============================] - 0s 29ms/step
Prediction: non emergency


In [34]:
if(pred=='emergency'):
  !python detect.py --source /content/drive/MyDrive/ambulance/test --weights /content/drive/MyDrive/last.pt --img 704 --save-txt --save-conf
else:
  print("Not an Emergency Vehicle: THE SIGNAL WILL STAY AT RED")

Not an Emergency Vehicle: THE SIGNAL WILL STAY AT RED
